In [1]:
!pip install unidecode
!pip install word2number
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5566 sha256=0c72015bf0e286308e5a032c422e193d3b807b72f0f20e33566d97a873b03713
  Stored in directory: /root/.cache/pip/wheels/84/ff/26/d3cfbd971e96c5aa3737ecfced81628830d7359b55fbb8ca3b
Successfully built word2number
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 20.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled h

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import re
import string
import unidecode
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
# import gensim.downloader as api
# from pycontractions import Contractions
from word2number import w2n
import json
import spacy
from transformers import AutoTokenizer
from datasets import load_dataset

import numpy as np
import nltk
import string
import random

Text Processing:

Text normalization (e.g., lowercasing, punctuation removal)
Tokenization and lemmatization
Handling of slang and abbreviations specific to mental health discussions

In [4]:
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
dataset = load_dataset("Amod/mental_health_counseling_conversations")
print(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response'],
        num_rows: 3512
    })
})


In [5]:
raw_train = dataset["train"]
print(raw_train)

Dataset({
    features: ['Context', 'Response'],
    num_rows: 3512
})


In [6]:
# print(raw_train.info)
print(raw_train[0]['Context'] + '\n')
print(raw_train[0]['Response'] + '\n')
print(raw_train.features)

I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?

If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media.  Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is somehow terrible.Bad feelings are part

In [7]:
train_df = dataset["train"].to_pandas()
train_df.head(1)

,Context,Response
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb..."


In [8]:
indices = train_df[train_df['Context'].str.contains('Advice on what to do?', case=False)].index

print(indices)

Index([1675, 2197], dtype='int64')


In [9]:
# idxs = [1675, 2197]
# selected_rows = train_df.loc[indices]
# print(selected_rows)

indices_to_display = train_df.sample(n=30).index

# Display the contents of the selected rows from the 'Context' column
selected_rows = train_df.loc[indices_to_display, 'Context']
print(selected_rows)

file_path = 'randomly_selected_rows.txt'

# Write the selected rows to the text file
with open(file_path, 'w') as file:
    for row in selected_rows:
        file.write(row + '\n')

print(f"Selected rows have been written to '{file_path}'.")

3353    I need help dealing with stress. How can I han...
113     Every winter I find myself getting sad because...
125     Every winter I find myself getting sad because...
1920    How do you know you have the right therapist f...
1043    Any time things get heated, the past gets brou...
2657    I would like to be able to have more positive ...
2678    I am in my mid 30s. I feel I have used only 5%...
1860    After he got home from the hospital he was ang...
2075    I've gone to a couple therapy sessions so far ...
2262    In 2008 my former husband of 14 years walked a...
2810    We have been together over a year. We spend ti...
2006    I have so many issues to address. I have a his...
2276    I'm going through a divorce with my wife of th...
1865    After he got home from the hospital he was ang...
3330    My wife and I got separated because I asked ab...
1145    Ever since my ex-boyfriend and I broke up, I c...
1981    I have so many issues to address. I have a his...
285     I've h

In [10]:
print(indices_to_display)

Index([3353,  113,  125, 1920, 1043, 2657, 2678, 1860, 2075, 2262, 2810, 2006,
       2276, 1865, 3330, 1145, 1981,  285,  498, 1516,  733, 1266,  169, 1452,
         53, 2688, 2867, 1009, 2324, 3139],
      dtype='int64')


In [11]:
contractions_dict = {
    "ain't": "is not", "aren't": "are not", "can't": "cannot", "couldn't": "could not",
    "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not",
    "hasn't": "has not", "haven't": "have not", "he'd": "he would", "he'll": "he will",
    "he's": "he is", "i'm": "i am", "I'd": "I would", "I'll": "I will", "I'm": "I am", "I've": "I have",
    "isn't": "is not", "it'd": "it would", "it'll": "it will", "it's": "it is",
    "let's": "let us", "mightn't": "might not", "mustn't": "must not", "shan't": "shall not",
    "she'd": "she would", "she'll": "she will", "she's": "she is", "shouldn't": "should not",
    "that's": "that is", "there's": "there is", "they'd": "they would", "they'll": "they will",
    "they're": "they are", "they've": "they have", "we'd": "we would", "we're": "we are",
    "we've": "we have", "weren't": "were not", "what'll": "what will", "what's": "what is",
    "who's": "who is", "won't": "will not", "wouldn't": "would not", "you'd": "you would",
    "you'll": "you will", "you're": "you are", "you've": "you have"
}

def expand_contractions(text, contractions_dict):
    pattern = re.compile(r'\b(' + '|'.join(contractions_dict.keys()) + r')\b')
    return pattern.sub(lambda x: contractions_dict[x.group()], text)

train_df['Context'] = train_df['Context'].apply(lambda x: expand_contractions(x, contractions_dict))
train_df['Response'] = train_df['Response'].apply(lambda x: expand_contractions(x, contractions_dict))
train_df.head(1)

,Context,Response
0,I am going through some things with my feeling...,"If everyone thinks you are worthless, then may..."


In [12]:
df = train_df
# convert text to lowercase
df['Context'] = df['Context'].str.lower()
df['Response'] = df['Response'].str.lower()
df.head(1)

,Context,Response
0,i am going through some things with my feeling...,"if everyone thinks you are worthless, then may..."


In [13]:
nltk.download('punkt')
# Strip punctuation
df['Context'] = df['Context'].str.translate(str.maketrans('', '', string.punctuation))
df['Response'] = df['Response'].str.translate(str.maketrans('', '', string.punctuation))

# tokenize text
df['Context'] = df['Context'].apply(word_tokenize)
df['Response'] = df['Response'].apply(word_tokenize)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [14]:
# remove non-alphabetic characters
df['Context'] = df['Context'].apply(lambda x: [word for word in x if word.isalpha()])
df['Response'] = df['Response'].apply(lambda x: [word for word in x if word.isalpha()])

In [15]:
# remove stop-words
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
df['Context'] = df['Context'].apply(lambda x: [word for word in x if word not in stop_words])
df['Response'] = df['Response'].apply(lambda x: [word for word in x if word not in stop_words])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [16]:
# lemmatization
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
df['Context'] = df['Context'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
df['Response'] = df['Response'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [17]:
df.dropna(inplace=True)

In [18]:
df.head()

,Context,Response
0,"[going, thing, feeling, barely, sleep, nothing...","[everyone, think, worthless, maybe, need, find..."
1,"[going, thing, feeling, barely, sleep, nothing...","[hello, thank, question, seeking, advice, feel..."
2,"[going, thing, feeling, barely, sleep, nothing...","[first, thing, would, suggest, getting, sleep,..."
3,"[going, thing, feeling, barely, sleep, nothing...","[therapy, essential, feeling, depressed, worth..."
4,"[going, thing, feeling, barely, sleep, nothing...","[first, want, let, know, alone, feeling, alway..."


In [19]:


# # Load Spacy model
# nlp = spacy.load('en_core_web_sm')

# def preprocess_text(text):
#     # Normalize text by lowercasing and removing punctuation
#     text = text.lower()
#     text = text.translate(str.maketrans('', '', string.punctuation))

#     # Tokenization and lemmatization using Spacy
#     doc = nlp(text)
#     lemmatized = [token.lemma_ for token in doc if not token.is_stop]

#     return lemmatized

# # Apply preprocessing to the dataset
# processed_data = [{'Context': preprocess_text(item['Context']), 'Response': preprocess_text(item['Response'])} for item in data]

In [20]:
# def clean_text(txt):
#     txt = txt.lower()
#     txt = re.sub(r"i'm", "i am", txt)
#     txt = re.sub(r"i'd", "i would", txt)
#     txt = re.sub(r"he's", "he is", txt)
#     txt = re.sub(r"she's", "she is", txt)
#     txt = re.sub(r"that's", "that is", txt)
#     txt = re.sub(r"what's", "what is", txt)
#     txt = re.sub(r"where's", "where is", txt)
#     txt = re.sub(r"\'ll", "will", txt)
#     txt = re.sub(r"\'ve", "have", txt)
#     txt = re.sub(r"\'re", "are", txt)
#     txt = re.sub(r"\'d", "would", txt)
#     txt = re.sub(r"won't", "will not", txt)
#     txt = re.sub(r"can't", "can not", txt)
#     txt = re.sub(r"[^\w\s]", "", txt)
#     return txt


Intent Classification:

Development of a classification model to discern user intents such as seeking advice, asking for references, or needing urgent help.
Training the model using supervised learning techniques with labeled examples from the datasets.

In [21]:
df['intent'] = None

# defining small sample of intentions
# [   4, 1024, 3170, 2895, 1486, 2879,   88, 1513, 2166, 1171, 1530,  891,
    #    1438, 1234,  745, 2989, 2074,  456,  337, 3073, 2873, 1564,  735, 3469,
    #    1546, 2793,   61, 2772, 2462, 3063]
df.at[4, 'intent'] = 'Coping with depression or sadness'
df.at[1024, 'intent'] = 'Managing anger or frustration'
df.at[3170, 'intent'] = 'Exploring spiritual or existential concerns'
df.at[2895, 'intent'] = 'Addressing relationship issues or conflicts'
df.at[1486, 'intent'] = 'Addressing relationship issues or conflicts'
df.at[2879, 'intent'] = 'Exploring spiritual or existential concerns'
df.at[88, 'intent'] = 'Managing symptoms of a mental health condition'
df.at[1513, 'intent'] = 'Addressing relationship issues or conflicts'
df.at[2166, 'intent'] = 'Coping with depression or sadness'
df.at[1171, 'intent'] = 'Exploring identity or self-discovery'
df.at[1530, 'intent'] = 'Addressing relationship issues or conflicts'
df.at[891, 'intent'] = 'Addressing work-related stress or burnout'
df.at[1438, 'intent'] = 'Addressing relationship issues or conflicts'
df.at[1234, 'intent'] = 'Addressing relationship issues or conflicts'
df.at[745, 'intent'] = 'Improving self-esteem or self-confidence'
df.at[2989, 'intent'] = 'Managing symptoms of a mental health condition'
df.at[2074, 'intent'] = 'Managing symptoms of anxiety'
df.at[456, 'intent'] = 'Managing social phobia'
df.at[337, 'intent'] = 'Managing social phobia'
df.at[3073, 'intent'] = 'Managing symptoms of anxiety'
df.at[2873, 'intent'] = 'Exploring spiritual or existential concerns'
df.at[1564, 'intent'] = 'Addressing relationship issues or conflicts'
df.at[735, 'intent'] = 'Improving self-esteem or self-confidence'
df.at[3469, 'intent'] = 'Seeking help for fears'
df.at[1546, 'intent'] = 'Addressing relationship issues or conflicts'


In [22]:
df.head(10)

,Context,Response,intent
0,"[going, thing, feeling, barely, sleep, nothing...","[everyone, think, worthless, maybe, need, find...",None
1,"[going, thing, feeling, barely, sleep, nothing...","[hello, thank, question, seeking, advice, feel...",None
2,"[going, thing, feeling, barely, sleep, nothing...","[first, thing, would, suggest, getting, sleep,...",None
3,"[going, thing, feeling, barely, sleep, nothing...","[therapy, essential, feeling, depressed, worth...",None
4,"[going, thing, feeling, barely, sleep, nothing...","[first, want, let, know, alone, feeling, alway...",Coping with depression or sadness
5,"[going, thing, feeling, barely, sleep, nothing...","[heck, sure, thing, hunfeelings, depression, d...",None
6,"[going, thing, feeling, barely, sleep, nothing...","[exhibiting, specific, trait, particular, temp...",None
7,"[going, thing, feeling, barely, sleep, nothing...","[intense, depression, liar, sometimes, depress...",None
8,"[going, thing, feeling, barely, sleep, nothing...","[sound, like, may, putting, last, wrote, want,...",None
9,"[going, thing, feeling, barely, sleep, nothing...","[must, really, difficult, experience, going, r...",None


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

# join text and then vectorize text
df['context_str'] = df['Context'].apply(lambda x: ' '.join(x))
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['context_str']).toarray()

In [24]:
#tfidf score for the first conversation
print(X[0, :])

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.13561461 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.18277524
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.25045444 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0. 

In [25]:
from sklearn.cluster import KMeans

# clustering
unique_labels = df['intent'].unique()
num_clusters = len(unique_labels)
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
df['cluster'] = kmeans.fit_predict(X)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [26]:
#assigning intents based on clusters
new_intent={}
kmeans_clusters=df.groupby('cluster')
for cluster,cluster_df in kmeans_clusters:
    cluster_count=cluster_df['intent'].value_counts()
    if not cluster_count.empty:
        max_cluster_intent=cluster_count.idxmax()
        new_intent[cluster]=max_cluster_intent
    else:
        new_intent[cluster] = 'Unknown'
df['new_intent']=df['cluster'].map(new_intent)

df.head()

,Context,Response,intent,context_str,cluster,new_intent
0,"[going, thing, feeling, barely, sleep, nothing...","[everyone, think, worthless, maybe, need, find...",None,going thing feeling barely sleep nothing think...,3,Coping with depression or sadness
1,"[going, thing, feeling, barely, sleep, nothing...","[hello, thank, question, seeking, advice, feel...",None,going thing feeling barely sleep nothing think...,3,Coping with depression or sadness
2,"[going, thing, feeling, barely, sleep, nothing...","[first, thing, would, suggest, getting, sleep,...",None,going thing feeling barely sleep nothing think...,3,Coping with depression or sadness
3,"[going, thing, feeling, barely, sleep, nothing...","[therapy, essential, feeling, depressed, worth...",None,going thing feeling barely sleep nothing think...,3,Coping with depression or sadness
4,"[going, thing, feeling, barely, sleep, nothing...","[first, want, let, know, alone, feeling, alway...",Coping with depression or sadness,going thing feeling barely sleep nothing think...,3,Coping with depression or sadness


In [27]:
#w2v embedding
from gensim.models import Word2Vec
full_text_token=df['context_str'].apply(lambda x: x.split())
w2v_model=Word2Vec(full_text_token,vector_size=100,window=5,min_count=1,workers=4)
w2v_model.save("word2vec_model.model")

Model Development:

Research and selection between Seq2Seq, Transformer, or LSTM-based models for generating responses.
Implementation and training of the chosen model.
Evaluation of model performance and tuning.

In [28]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


X=df['context_str'].values
y = df['Response'].values

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_tokenized = tokenizer.texts_to_sequences(X)

max_sequence_length = 100
X_padded = pad_sequences(X_tokenized, maxlen=max_sequence_length, padding='post', truncating='post')
'''
# Split data into training, validation, and testing sets
X_train, X_val_test, y_train, y_val_test = train_test_split(X_padded, y_tokenized, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)
'''

'\n# Split data into training, validation, and testing sets\nX_train, X_val_test, y_train, y_val_test = train_test_split(X_padded, y_tokenized, test_size=0.2, random_state=42)\nX_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)\n'

In [30]:
tokenizer_target = Tokenizer()
tokenizer_target.fit_on_texts(y)
y_tokenized = tokenizer_target.texts_to_sequences(y)
max_sequence_length_target = max(len(sequence) for sequence in y_tokenized)
y_padded = pad_sequences(y_tokenized, maxlen=max_sequence_length_target, padding='post', truncating='post')

X_train, X_val_test, y_train, y_val_test = train_test_split(X_padded, y_padded, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.initializers import Constant

embedding_dim = 100
vocab_size = len(tokenizer.word_index) + 1
num_classes=len(df['Response'].apply(lambda x: ' '.join(x)).unique())

word2vec_model = Word2Vec.load("word2vec_model.model")


embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]


lstm_model = Sequential([
    Embedding(vocab_size, embedding_dim, embeddings_initializer=Constant(embedding_matrix), input_length=max_sequence_length, trainable=False),
    LSTM(128,dropout=0.2,recurrent_dropout=0.2,return_sequences=True),
    LSTM(128),
    Dense(max_sequence_length_target, activation='softmax')
])

lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
'''

model = Sequential([
    Embedding(input_dim=embedding_matrix.shape[0],
              output_dim=embedding_matrix.shape[1],
              embeddings_initializer=Constant(embedding_matrix),
              input_length=max_sequence_length,
              trainable=False),  # Optionally, set trainable=False to freeze the embeddings
    LSTM(128, dropout=0.5, recurrent_dropout=0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])
'''

lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/10
88/88 [==============================] - 49s 495ms/step - loss: 526624.0625 - accuracy: 0.0110 - val_loss: 530068.1875 - val_accuracy: 0.0114
Epoch 2/10
88/88 [==============================] - 47s 532ms/step - loss: 576973.6875 - accuracy: 0.0100 - val_loss: 608638.8125 - val_accuracy: 0.0114
Epoch 3/10
88/88 [==============================] - 69s 782ms/step - loss: 642413.3125 - accuracy: 0.0125 - val_loss: 697652.9375 - val_accuracy: 0.0114
Epoch 4/10
88/88 [==============================] - 44s 496ms/step - loss: 738436.5000 - accuracy: 0.0125 - val_loss: 798054.9375 - val_accuracy: 0.0114
Epoch 5/10
88/88 [==============================] - 45s 508ms/step - loss: 853678.8750 - accuracy: 0.0125 - val_loss: 899336.1875 - val_accuracy: 0.0114
Epoch 6/10
88/88 [==============================] - 43s 487ms/step - loss: 934403.2500 - accuracy: 0.0117 - val_loss: 1000233.6875 - val_accuracy: 0.0114
Epoch 7/10
88/88 [==============================] - 44s 500ms/step - loss: 104391

Response Generation and Information Retrieval:

Generating appropriate responses based on the classified intent and model outputs.
Integrating external API or database queries to fetch and provide real-time counselor references and additional mental health resources.

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import OneHotEncoder

def preprocessing_text(input_text):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(input_text)
    input_sequences = tokenizer.texts_to_sequences(input_text)

    max_sequence_length = max(len(seq) for seq in input_sequences)
    padded_input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='post')

     # Word2Vec embedding
    input_text_embeddings = []
    for text in input_text:
        text_embedding = []
        for word in text.split():
            if word in w2v_model.wv:
                text_embedding.append(w2v_model.wv[word])
        input_text_embeddings.append(np.mean(text_embedding, axis=0) if text_embedding else np.zeros_like(w2v_model.wv["word"]))
    input_text_embeddings = np.array(input_text_embeddings)

    vocabulary_size = len(tokenizer.word_index) + 1

    return padded_input_sequences,input_text_embeddings, vocabulary_size


def decode_output_sequence(output_sequence, tokenizer, max_length=20):
    predicted_words = []
    for _ in range(max_length):
        token_index = np.argmax(output_sequence)
        predicted_word = tokenizer.index_word.get(token_index, "")
        predicted_words.append(predicted_word)
        if predicted_word == "<end>":
            break

    return " ".join(predicted_words)



input_text = ["need help"]
padded_input_sequences,input_text_embeddings, vocabulary_size = preprocessing_text(input_text)
input_sequence_padded = pad_sequences(padded_input_sequences, maxlen=max_sequence_length)
predicted_output_sequence = lstm_model.predict(input_text_embeddings)

print("Input Text:", input_text)
predicted_output_word = decode_output_sequence(predicted_output_sequence, tokenizer)
print("Predicted Output:", predicted_output_word)
